# Data Cleaning

Having acquired the data, this notebook will clean and preprocess text and extract new metadata.

In [2]:
import pandas as pd
import numpy as np
import re
from functools import reduce

In [3]:
df = pd.read_pickle('../data/interim/aggregated.pkl')

In [4]:
def find_handles(text):
    r = re.compile('@[^\s]+')
    return re.findall(r, text)

def find_hashtags(text):
    r = re.compile('#[^\s]+')
    return re.findall(r, text)

In [5]:
df['Tweeted At'] = df['Contents'].map(find_handles)

In [6]:
df['Hashtags'] = df['Contents'].map(find_hashtags)

### Create Hashtags and Tweeted At DataFrame

In [29]:
def make_gen(key, series):
    for i, keys in series.iteritems():
        if key in keys:
            yield i

In [7]:
hashtags = {}

for i, tags in df['Hashtags'].iteritems():
    for tag in tags:
        hashtags[tag] = hashtags.setdefault(tag, []) + [i]

In [8]:
d = {'Hashtag': [k for k in hashtags.keys()], 'Indices': [v for v in hashtags.values()]}
df_hashtags = pd.DataFrame(d)

In [9]:
df_hashtags.to_pickle('../data/interim/hashtag_indices.pkl')

In [10]:
df_mphandles = pd.read_csv('../data/interim/female_MP_handles.csv')
df_mphandles['twitter_handles'] = df_mphandles['twitter_handles'].map(lambda x: '@' + x)
df_mphandles.head()

,twitter_handles
0,@Alison_McGovern
1,@alisonthewliss
2,@amandamilling
3,@AmberRuddHR
4,@andreajenkyns


In [26]:
df_mphandles['Mentions'] = df_mphandles['twitter_handles'].map(lambda x: [i for i in make_gen(x, df['Tweeted At'])])

df_mphandles.to_pickle('../data/interim/tweeted_at.pkl')

In [27]:
df_mphandles['Counts'] = df_mphandles['Mentions'].map(lambda x : len(x))

In [30]:
df_mphandles['Counts'].describe()

count       176.000000
mean      12633.897727
std       29954.171118
min          12.000000
25%        1631.500000
50%        4818.500000
75%       11193.750000
max      277115.000000
Name: Counts, dtype: float64